In [1]:
from fastai.vision.all import *
from scipy.io import loadmat

In [2]:
datapath = Path("stanford-dogs-dataset/")

In [3]:
list(datapath.ls())

[Path('stanford-dogs-dataset/train_list.mat'),
 Path('stanford-dogs-dataset/sample_train.csv'),
 Path('stanford-dogs-dataset/images'),
 Path('stanford-dogs-dataset/test_list.mat'),
 Path('stanford-dogs-dataset/file_list.mat'),
 Path('stanford-dogs-dataset/annotations')]

In [5]:
test_filenames = [o[0][0] for o in loadmat(datapath/'test_list.mat')['file_list']]
train_filenames = [o[0][0] for o in loadmat(datapath/'train_list.mat')['file_list']]
test_filenames = [o[0][0] for o in loadmat(datapath/'test_list.mat')['file_list']]

In [20]:
len(train_filenames), len(test_filenames)

(12000, 8580)

In [13]:
train_labels = L(train_filenames).map(lambda o: o.split("/")[0])

In [14]:
train_df = pd.DataFrame({"filenames":train_filenames, "labels": train_labels})

In [15]:
test_labels = L(test_filenames).map(lambda o: o.split("/")[0])

In [16]:
test_df = pd.DataFrame({"filenames":test_filenames, "labels": test_labels})

In [17]:
train_df.to_csv(datapath/"train.csv", index=False)
test_df.to_csv(datapath/"test.csv", index=False)

In [19]:
train_df.shape, test_df.shape

((12000, 2), (8580, 2))

### Subsample Train (40 train, 10 valid per class) - 4800/1200

In [79]:
class2filename = dict(train_df.groupby("labels")['filenames'].apply(lambda o: np.random.choice(o, size=50, replace=False)))

In [80]:
sample_train_df = []
for c in class2filename:
    for v in class2filename[c]:
        sample_train_df.append((c,v))

In [81]:
sample_train_df = pd.DataFrame(sample_train_df, columns=['label', 'filename'])

In [92]:
valid_filenames = sample_train_df.groupby("label").head(10)['filename']
sample_train_df['split'] = 'train'
sample_train_df.loc[sample_train_df.filename.isin(valid_filenames), 'split'] = 'valid'

In [95]:
sample_train_df.to_csv(datapath/"sample_train.csv", index=False)